In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn import cross_validation
from sklearn.metrics import roc_auc_score as auc
from sklearn.preprocessing import StandardScaler

In [ ]:
trainDataFrame = pd.read_csv('./data/train.csv')

# remove constant columns
colsToRemove = []
for col in trainDataFrame.columns:
    if trainDataFrame[col].std() == 0:
        colsToRemove.append(col)

trainDataFrame.drop(colsToRemove, axis=1, inplace=True)

# remove duplicate columns
colsToRemove = []
columns = trainDataFrame.columns
for i in range(len(columns)-1):
    v = trainDataFrame[columns[i]].values
    for j in range(i+1,len(columns)):
        if np.array_equal(v,trainDataFrame[columns[j]].values):
            colsToRemove.append(columns[j])

trainDataFrame.drop(colsToRemove, axis=1, inplace=True)

trainLabels = trainDataFrame['TARGET']
trainFeatures = trainDataFrame.drop(['ID','TARGET'], axis=1)

In [ ]:
X_train, X_valid, y_train, y_valid = cross_validation.train_test_split(
    trainFeatures, trainLabels, test_size=0.3, random_state=1)

In [ ]:
# scale data:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Don't cheat - fit only on training data
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled)

X_valid_scaled = scaler.transform(X_valid)
X_valid_scaled = pd.DataFrame(X_valid_scaled)


In [ ]:
import sys
sys.path.insert(0, "/Users/gokhan/libs")
from Evio.evolver import RandomConceptEvolver3

In [ ]:
# 10K concepts on NON-SCALED data & NO EVALUATION OF FEATURES w following attrs
rce3 = RandomConceptEvolver3(n_concepts=10000, n_gens=500, random_state=1, mate_selection="roulette",
                             eval_features_every_n_gens=0,
                             eliminate_perc_features=0.0, eval_metric="auc",
                             n_attrs_per_feature=[(1, 0.25), (2, 0.25), (3, 0.25), (4, 0.25)])
rce3.fit(X_train, y_train, 2, n_folds=10) # auc

In [ ]:
rce3.fit(X_train, y_train, 2, n_folds=10, warm_start=True)

In [ ]:
auc(y_valid, rce3.predict(X_valid))

In [ ]:
np.unique(rce3.best_individual, return_counts=True)

In [ ]:
rce3.eval_features_every_n_gens = 400
rce3._eval_features()
rce3.eval_features_every_n_gens = 0

In [ ]:
rce3._feature_scores

In [ ]:
rce3._feature_schema

In [ ]:
# DO NOT PLOT!!! IT CRASHES THE KERNEL!!!
# rce3.plot()